# Russian language dataset

In [75]:
from fastai.basics import *
from multiprocessing import Pool
import regex as re
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

In [76]:
librusec = '/home/u/nas/librusec/lib.rus.ec'
tmp = './tmp/fb2'
data = Path('../data/full')

In [77]:
!mkdir ../data
!mkdir ../data/full
!mkdir ../data/classic
!mkdir tmp
!mkdir tmp/fb2
!mkdir tmp/txt

mkdir: cannot create directory ‘../data’: File exists
mkdir: cannot create directory ‘../data/full’: File exists
mkdir: cannot create directory ‘../data/classic’: File exists
mkdir: cannot create directory ‘tmp’: File exists


### unpack zips

In [5]:
zips = get_files(librusec, '.zip')

In [6]:
def unpack(fn):
    !unzip {fn} -d {tmp} >>/dev/null

In [ ]:
for _ in progress_bar(Pool(64).imap_unordered(unpack, zips), len(zips)):
    pass

### convert fb2 to txt

In [7]:
fbs = get_files(tmp, '.fb2')

In [8]:
def convert_fb2(fn):
    !xsltproc FB2_2_txt.xsl {fn} > {str(fn).replace('.fb2','.txt').replace('/fb2','/txt')} 2>>/dev/null

In [ ]:
for _ in progress_bar(Pool(64).imap_unordered(convert_fb2, fbs), len(fbs)):
    pass

### more data

In [106]:
!rm -R /share/CE_CACHEDEV1_DATA/data/classic_lit/*
!cp -R /share/CE_CACHEDEV1_DATA/data/downloads/txt /share/CE_CACHEDEV1_DATA/data/classic_lit/

rm: cannot remove '/share/CE_CACHEDEV1_DATA/data/classic_lit/*': No such file or directory
cp: cannot stat '/share/CE_CACHEDEV1_DATA/data/downloads/txt': No such file or directory


In [113]:
fbs = get_files('/home/u/nas/classic_lit/txt/', '.fb2', recurse=True); len(fbs)

0

In [108]:
!rm -R /home/u/nas/classic_lit/out

rm: cannot remove '/home/u/nas/classic_lit/out': No such file or directory


In [109]:
path_out = Path('/home/u/nas/classic_lit/out'); path_out.mkdir(exist_ok = True)

In [110]:
for fn in fbs:
    nn = (str(fn.name)
        .replace(' ','.')
        .replace('_quot;','.')
        .replace('!','.')
        .replace(',','.')
        .replace('(','.')
        .replace(')','.')
        .replace('\xa0','.')
         )
    shutil.move(fn, path_out/nn)

In [118]:
fbs = get_files(path_out, '.fb2'); len(fbs)

832

In [114]:
!rm ../data/classic/*

rm: cannot remove '../data/classic/*': No such file or directory


In [115]:
data = Path('../data/classic')

In [119]:
def convert_fb2(fn):
    name = str(fn.name).replace('.fb2','.txt')
    !xsltproc FB2_2_txt.xsl {fn} > {data/name} 2>>/dev/null

In [120]:
for _ in progress_bar(Pool(64).imap_unordered(convert_fb2, fbs), len(fbs)):
    pass

In [121]:
!rm ../data/classic/*месяцеслов*

### filter txts

In [4]:
txts = get_files('./tmp/txt', '.txt')

In [5]:
len(txts)

428809

In [6]:
# test
fn = txts[0]
match = re.compile('(.|\s)\\1\\1+')
with open(fn, 'r') as f:
    lines = f.read()
lines = 'asdf aaaa dddddfffff' + lines
lines = match.sub(r'\1'*3, lines)             
lines[:1000]

'asdf aaa dddfffУбийцы (Мир пауков - 6)\nКолин Уилсон\n\n\nУилсон Колин\n\nУбийцы (Мир пауков - 6)\n\n\nКолин Уилсон \n\nмир пауков \n\nфантастический роман \n\n"УБИЙЦЫ" \n\nперевод с английского АЛЕКСАНДРА ШАБРИНА \n\nНезадолго до рассвета Найл проснулся от холода - такого пронизывающего, что вспомнилась ночь в пустыне. Простыни сбились вокруг лица, а от дыхания на одеяле образовался кружок влаги. Эту комнату Найл выбрал потому, что она выходила окнами на восток, а ему нравилось просыпаться вместе с солнцем. Сегодня солнца не было. Рассвет проплавлялся сквозь серую туманную пелену, пока комнату не залил холодный, ровный свет. И птиц не было слышно. \n\nЧто-то в этой странной тишине настораживало. Найл, ступая по мягкому шерстяному половику, прошел по комнате к окну, и глазам неожиданно открылся белый пейзаж. Белые крыши, словно горные вершины терялись в перламутрово-сером небе, и огромная площадь была как ковром покрыта той же безликой белизной. Даже вон углы оконных рам прихватило: с

In [7]:
# test
detect(lines)

'ru'

In [8]:
# test
expr = re.compile('([^\n]{150,})([.] )([^\n]{150,})')
while expr.search(lines):
    lines = expr.sub(r'\g<1>.\n\g<3>', lines, 1)
lines[:1000]

'asdf aaa dddfffУбийцы (Мир пауков - 6)\nКолин Уилсон\n\n\nУилсон Колин\n\nУбийцы (Мир пауков - 6)\n\n\nКолин Уилсон \n\nмир пауков \n\nфантастический роман \n\n"УБИЙЦЫ" \n\nперевод с английского АЛЕКСАНДРА ШАБРИНА \n\nНезадолго до рассвета Найл проснулся от холода - такого пронизывающего, что вспомнилась ночь в пустыне. Простыни сбились вокруг лица, а от дыхания на одеяле образовался кружок влаги.\nЭту комнату Найл выбрал потому, что она выходила окнами на восток, а ему нравилось просыпаться вместе с солнцем. Сегодня солнца не было. Рассвет проплавлялся сквозь серую туманную пелену, пока комнату не залил холодный, ровный свет. И птиц не было слышно. \n\nЧто-то в этой странной тишине настораживало. Найл, ступая по мягкому шерстяному половику, прошел по комнате к окну, и глазам неожиданно открылся белый пейзаж.\nБелые крыши, словно горные вершины терялись в перламутрово-сером небе, и огромная площадь была как ковром покрыта той же безликой белизной. Даже вон углы оконных рам прихватило:

In [9]:
re.sub(r'(?=[^ ])([\W])([\w])', r'\g<1> \g<2>', lines)[:1000]

'asdf aaa dddfffУбийцы ( Мир пауков - 6)\n Колин Уилсон\n\n\n Уилсон Колин\n\n Убийцы ( Мир пауков - 6)\n\n\n Колин Уилсон \n\n мир пауков \n\n фантастический роман \n\n" УБИЙЦЫ" \n\n перевод с английского АЛЕКСАНДРА ШАБРИНА \n\n Незадолго до рассвета Найл проснулся от холода - такого пронизывающего, что вспомнилась ночь в пустыне. Простыни сбились вокруг лица, а от дыхания на одеяле образовался кружок влаги.\n Эту комнату Найл выбрал потому, что она выходила окнами на восток, а ему нравилось просыпаться вместе с солнцем. Сегодня солнца не было. Рассвет проплавлялся сквозь серую туманную пелену, пока комнату не залил холодный, ровный свет. И птиц не было слышно. \n\n Что- то в этой странной тишине настораживало. Найл, ступая по мягкому шерстяному половику, прошел по комнате к окну, и глазам неожиданно открылся белый пейзаж.\n Белые крыши, словно горные вершины терялись в перламутрово- сером небе, и огромная площадь была как ковром покрыта той же безликой белизной. Даже вон углы оконных

In [10]:
# this will take time, bcs langdetect fails on multithreading
for fn in progress_bar(txts):
    with open(fn, 'r') as f:
        lines = f.read()
        try:
            if len(lines) > 1e+4 and detect(lines) == 'ru':
                with open(f'{data}/{fn.name}', 'w') as c:
                    c.write(lines)
        except LangDetectException as e:
            pass

In [123]:
txts = get_files(data, '.txt')

In [124]:
#txts += ['../data/poetry_base.txt']

In [134]:
with open('../data/poetry_base.txt', 'r') as f:
    lines = f.readlines()
    
split = int(len(lines)*0.95)

with open('../data/poetry_dry.txt', 'w') as f:
    f.writelines(lines[:split])

with open('../data/poetry_eval.txt', 'w') as f:
    f.writelines(lines[split:])

In [126]:
# Add space before each word. It's not really nesessary. 
# It just makes encoding a bit more meaningful to the model and the text smaller(after encoding).

def process_fn(fn):
    match = re.compile(r'(?=[^ ])([\W])([\w])')
    match2 = re.compile('(.|\s)\\1\\1+')
    with open(fn, 'r') as f:
        lines = f.read()
    if lines[0] != ' ': lines = ' ' + lines
    lines = match.sub(r'\g<1> \g<2>', lines)
    lines = match2.sub(r'\1'*3, lines)
    with open(fn, 'w') as c:
        c.write(lines)
        
for _ in progress_bar(Pool(64).imap_unordered(process_fn, txts), len(txts)):
    pass

### concat for vocab

Take smallest files to increase word diversity

In [22]:
txts = get_files(data, '.txt'); len(txts)

343067

In [23]:
for fn in txts:
    if os.path.getsize(fn) <= 1e+4: 
        os.remove(fn)

In [24]:
txts = get_files(data, '.txt'); len(txts)

343064

In [ ]:
fsorted = sorted(txts, key=lambda fn: os.path.getsize(fn))  

In [ ]:
sz=0
with open('./tmp/russian_corpus_for_vocab.txt', 'w') as c:
    for fn in fsorted:
        with open(fn, 'r') as f:
            sz += c.write(f.read() + '\n')
            if sz > 5e+9:
                break

### cache tokenization (optional)

In [25]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [26]:
from run_lm_finetuning import TextDataset
from sp_encoder import SPEncoder

In [27]:
txts = get_files(data, '.txt'); len(txts)

343064

In [28]:
def cache_fn(fn):
    tokenizer = SPEncoder.from_pretrained('../bpe/m50.model')
    TextDataset.process_file(fn, tokenizer, 1024)

In [29]:
for _ in progress_bar(Pool(32).imap_unordered(cache_fn, txts), len(txts)):
    pass